# 필요한 라이브러리 import

In [69]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

# 데이터 불러오기 및 확인

In [70]:
df = pd.read_csv("../input/netflix-shows/netflix_titles.csv")
df.head()

In [71]:
df.info()

In [72]:
df.nunique()

In [73]:
# 결측치 확인
df.isna().sum()

# 데이터 전처리

## Rating, duration 처리

In [74]:
df['rating'].unique()

In [75]:
df[df['rating'].isna()]

In [76]:
df['duration'].unique()

In [77]:
df[df['duration'].isna()]

4,3개뿐이니 일단 pass

## director, cast 처리

In [78]:
df = df.drop(['director', 'cast'], axis=1)
df.columns

결측치가 너무 많으므로 일단 제외

## date_added 처리

In [79]:
df[df['date_added'].isna()]

In [80]:
df = df[df['date_added'].notna()]
df

date_added의 결측치도 제외<br>
결측치가 없는 항목들만 따로 뽑아서 사용

## country 처리

In [81]:
df['country'] = df['country'].fillna(df['country'].mode()[0])
df

## 전처리 확인

In [82]:
df.isna().sum()

In [83]:
df.head(10)

## year_added, month_added 생성

`date_added`에서 year, month 값 추출

In [84]:
df['year_added'] = df['date_added'].apply(lambda x: x.split(" ")[-1])
df['year_added'].head()

In [85]:
df['month_added'] = df['date_added'].apply(lambda x: x.split(" ")[0])
df['month_added'].head()

In [86]:
df.head()

## rating 기준으로 TV 타겟층 생성

In [87]:
ratings_ages = {
    'TV-PG': 'Older Kids',
    'TV-MA': 'Adults',
    'TV-Y7-FV': 'Older Kids',
    'TV-Y7': 'Older Kids',
    'TV-14': 'Teens',
    'R': 'Adults',
    'TV-Y': 'Kids',
    'NR': 'Adults',
    'PG-13': 'Teens',
    'TV-G': 'Kids',
    'PG': 'Older Kids',
    'G': 'Kids',
    'UR': 'Adults',
    'NC-17': 'Adults'
}

In [88]:
df['target_ages'] = df['rating'].replace(ratings_ages)
df['target_ages'].unique()

In [89]:
df.head()

In [90]:
# 여러개의 나라들로 구성 -> 대표로 맨 앞의 나라 지정 
df['principal_country'] = df['country'].apply(lambda x:x.split(",")[0])
df['principal_country'].head()

## data type 설정

In [91]:
df.dtypes

In [92]:
# type - category
df['type'] = pd.Categorical(df['type'])
# target_ages - category (4 classes)
df['target_ages'] = pd.Categorical(df['target_ages'], categories=['Kids', 'Older Kids', 'Teens', 'Adults'])

# year added - integer
df['year_added'] = pd.to_numeric(df['year_added'])

In [93]:
df.dtypes

# 데이터 시각화

In [95]:
df.head()

In [103]:
movie_df = df[df['type'] == 'Movie']
show_df = df[df['type'] == 'TV Show']

show_df.head()

In [104]:
fig = px.pie(df['type'].value_counts().reset_index(), values='type', names='index')
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

영화의 비중이 TV show보다 크다.

In [122]:
def generate_rating_df(df):
    rating_df = df.groupby(['rating', 'target_ages']).agg({'show_id': 'count'}).reset_index()
    rating_df = rating_df[rating_df['show_id'] != 0]
    rating_df.columns = ['rating', 'target_ages', 'counts']
    rating_df = rating_df.sort_values('target_ages')
    return rating_df

In [123]:
rating_df = generate_rating_df(df)
fig = px.bar(rating_df, x='rating', y='counts', color='target_ages')
fig.show()

어린이들을 위한 컨텐츠보다는 어른들을 위한 컨텐츠가 더 많다.